<img src="../Денис/dataset/Безымянный.png" height="200" width="900"> 

# <center> Web Scrapping of the website www.B17.ru </center>

For midterm project, we have to collect a raw data from the website www.b17.ru which contains numerous psychologists' profiles that we need to scrap onto our file. Here is the code that stands for parsing all profiles with respective attributes such as city,price,demand and so on

In [40]:
from requests import get 
import pandas as pd
import numpy as np
import time
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import re
from tqdm import tqdm_notebook

In [25]:
## Function for city
def getCity(soup):
    try:
        q=soup.find('div',{'class':'status-grey'}).text
        city=re.findall('\г\. \w+\-\w+',q)[0]
        print(city)
    except: 
        q=soup.find('div',{'class':'status-grey'}).text
        city=re.findall('\г\. \w+',q)[0]

    return city    

In [26]:
## Function for price

def getPrice(soup):
    obj=soup.find('div',{'style':'margin: 10px 0 0; padding-left: 21px; position: relative;'})
    obj=obj.text
    x=re.search('Видео-консультация',obj)
    if (x!=None):
        return [str(s) for s in obj.split() if s.isdigit()][0]
    else:
        return np.nan

In [27]:
## Function for demand - doesnt work
def getProperties(soup,stats):
    try:
        obj= soup.find('div',{'style':'margin: 6px 0; font-size: 13px; text-align: center;'})
        qwe=obj.find('a',{'class':stats})
        qwe=qwe.text
        obj=[int(s) for s in qwe.split() if s.isdigit()]
    except: 
        obj=np.nan
    return obj

In [28]:
## Function for N publications
def publication(soup):
    try:
        pub=soup.find('div',{'class':'vkladki5 no-print'}).find('a',{'id':'vkl_article'}).find('span',{'class':'n'})
        pub=pub.text
    except:
        pub=0
    return pub  

In [29]:
def feedback(soup):
    try:
        fd=soup.find('div',{'class':'vkladki5 no-print'}).find('a',{'id':'vkl_otzyv'}).find('span',{'class':'n'})
        fd=fd.text
    except:
        fd=0
    return fd  

In [30]:
## Function for methods of work
def methods_work(soup):
    try:
        methods=soup.find('div',{'style':'margin: 20px 0;'}).text
        methods=re.split(r'\W+',methods)[3:]
        methods=' '.join(methods) #to convert from list to string
    except: methods=np.nan    
    return methods

In [31]:
## Function for marital status
def marital_status(soup):
    try:
        marital=soup.find_all('div',{'style':'margin: 10px 0;'})[2].text #several info with the same tags, parsed only MS (2)
        marital=re.split(r'\W+',marital)[2:]
        marital=' '.join(marital)
    except: marital=np.nan
    return marital

In [32]:
## Function for age
def age_con(soup):
    try:
        q=soup.find_all('div',{'style':'margin: 10px 0;'})[1].text
        q=re.findall('\d\d\d\d',q)
        q=''.join(q)
        q=2021-int(q)
    except: q=0    
    return q

In [33]:
## Function for education
def education(soup):
    edu=soup.find('div',{'id':'vuz'}).find('div',{'class':'vkladki_body_in vkladki_body_list'}).find('div',{'id':'vuz_list_s'}).text
    edu=list(edu.split("\n"))[:-1]
    return edu

We finished to create a separate functions that retrieve a specific information from each profile. Lets combine it into single function

In [34]:
def psych(sor):
    
    #Created html paper

    response = get(sor, headers={'User-Agent': UserAgent().chrome})
    html = response.content
    soup = BeautifulSoup(html,'html.parser')
    
    if not response.ok:
        # если сервер нам отказал, вернем статус ошибки 
        return response.status_code
    
    #Apply function for demand
    demand=soup.find('div',{'style':'margin: 6px 0; font-size: 13px; text-align: center;'}).find('a')
    demand=demand.text
    demand=re.findall("\d+",demand)[0]

    
    #Apply function for price
    price=getPrice(soup)
    
    #Extract city 
    city=getCity(soup)
    
    #Extract name
    name=soup.find('h1').text
    
    #Extract number of Publications 
    pub=publication(soup)
    
    #Extract number of Feedbacks
    otz=feedback(soup)
    
    #Extract description
    des= soup.find('div',{'class':'vkladki_body0'})
    des=des.find('p')
    des=des.text
    
    #Extract methods of work
    methods=methods_work(soup)
    
    #Extract marital status
    marital=marital_status(soup)
    
    #Extract age
    age=age_con(soup)
    
    #Extract education
    edu=education(soup)
    data_col={
        "name":name,
        "demand":demand,
        "price":price,
        "publications":pub,
        "otzivy":otz,
        "description":des,
        "city":city,
        "methods":methods,
        "marital_status":marital,
        "age":age,
        "education":edu
    }
    
    return data_col

In [23]:
## Creating a dataset of single observation
df=pd.DataFrame(columns=["name","demand","price","publications","otzivy","description","city","methods","age","education"])
data_col=psych('https://www.b17.ru/ivanivanovichkuzmin/')
df=df.append(data_col,ignore_index=True)
df

г. Вольно-Надеждинское


IndexError: list index out of range

In [35]:
#Page cycle
def pagelink(meme_page1):
    page_link='https://www.b17.ru/online/?page={}'.format(meme_page1)
    response1 = get(page_link, headers={'User-Agent': UserAgent().chrome})
    if not response1.ok:
        # если сервер нам отказал, вернем пустой лист для текущей страницы
        return [] 
    html1 = response1.content
    soup1 = BeautifulSoup(html1,'html.parser')
    x=soup1.find_all("a",{"class":"h"})[1:]
    lina=['https://www.b17.ru'+link.get("href") for link in x]
    return lina

In [36]:
#Retrieving n observations from website

df10=pd.DataFrame(columns=["name","demand","price","publications","otzivy","description","city","methods","age","education"])


for page in tqdm_notebook(range(5),desc='Pages'):
    links=pagelink(page)
    for links in tqdm_notebook(links,desc='Psy',leave=False):
        for i in range(5):
            try:
                data_col=psych(links)
                df10=df10.append(data_col,ignore_index=True)
                break
            except:
                print(links)
                continue
        time.sleep(3)        


<ipython-input-36-02d4511ab133>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for page in tqdm_notebook(range(5),desc='Pages'):


<ipython-input-36-02d4511ab133>:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for links in tqdm_notebook(links,desc='Psy',leave=False):


г. Санкт-Петербург
г. Санкт-Петербург
https://www.b17.ru/kurakina_tn/
https://www.b17.ru/kurakina_tn/
https://www.b17.ru/kurakina_tn/
https://www.b17.ru/kurakina_tn/
https://www.b17.ru/kurakina_tn/
https://www.b17.ru/myratovaanna/
https://www.b17.ru/myratovaanna/
https://www.b17.ru/myratovaanna/
https://www.b17.ru/myratovaanna/
https://www.b17.ru/myratovaanna/


г. Санкт-Петербург
г. Санкт-Петербург
https://www.b17.ru/kurakina_tn/
https://www.b17.ru/kurakina_tn/
https://www.b17.ru/kurakina_tn/
https://www.b17.ru/kurakina_tn/
https://www.b17.ru/kurakina_tn/
https://www.b17.ru/myratovaanna/
https://www.b17.ru/myratovaanna/
https://www.b17.ru/myratovaanna/
https://www.b17.ru/myratovaanna/
https://www.b17.ru/myratovaanna/


https://www.b17.ru/liudiam/
https://www.b17.ru/liudiam/
https://www.b17.ru/liudiam/
https://www.b17.ru/liudiam/
https://www.b17.ru/liudiam/


https://www.b17.ru/snegurovich/
https://www.b17.ru/snegurovich/
https://www.b17.ru/snegurovich/
https://www.b17.ru/snegurovich/
https://www.b17.ru/snegurovich/
г. Вольно-Надеждинское


г. Франкфурт-на
г. Санкт-Петербург
г. Санкт-Петербург



In [39]:
df10.tail()

,name,demand,price,publications,otzivy,description,city,methods,age,education,marital_status
39,Назаренко Ирина Сергеевна,375,2500,75,24,"Есть опыт длительной личной терапии, тщательно...",г. Валенсия,Личностно центрированная терапия Психосинтез Э...,45,[Академия Постдипломного Педагогического Образ...,замужем
40,Кириллова Софья Григорьевна,298,1800,71,57,"Психологическое благополучие - это состояние, ...",г. Санкт-Петербург,Гештальт терапия Когнитивно поведенческая тера...,0,[Российский государственный университет имени ...,замужем
41,Алёна Андреевна Самортова,611,3000,253,45,Провожу офлайн и онлайн консультирование по во...,г. Москва,Гипнотерапия Когнитивно поведенческая терапия ...,30,"[Частное учреждение ""Образовательная организац...",Замужем
42,Стегний Павел Анатольевич,140,1700,342,17,Специализация: панические атаки и ВСД; самооце...,г. Кременчуг,Нейро лингвистическое программирование Психоан...,57,[Полтавский национальный педагогический универ...,женат
43,Яковкина Юлия Александровна,207,4000,185,7,"Я - магистр психологии, аспирант факультета Пс...",г. Санкт-Петербург,Когнитивная терапия Когнитивно поведенческая т...,34,"[РГПУ им. А.И. Герцена, Диплом Номинанта ХХ На...",замужем


In [49]:
for link in tqdm_notebook(lina):
    try:
        data_col=psych(link)
        df10=df10.append(data_col,ignore_index=True)
    except: 
        print(link)

<ipython-input-49-b380bc4f2659>:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for link in tqdm_notebook(lina):


г. Вольно-Надеждинское
г. Франкфурт-на



In [102]:
df10.to_csv('data1.csv',index=False,encoding='UTF-8')